In [ ]:
from module.prepare import *
from itertools import product
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import ParameterGrid
import redis

def scoreFunction(result):
    scores = {}

#     for result in cv_results:
    acc = metrics.accuracy_score(result[0],result[1])
    auc = metrics.roc_auc_score(result[0],result[1])
    scores['acc'] = acc
    scores['auc'] = auc
    
    return scores

#### LGB

In [ ]:
def LGBTuning(Xtrain,Xtest,Ytrain,Ytest,new_params=None,default=False,replace=False,tuning=True):
    
    clf = lgb.LGBMClassifier(objective='cross_entropy', ### {cross_entropy, binary}
                             silent=False,
                             verbose=1,
                             random_state=seed,
                             n_jobs=12,
#                              class_weight
                            )
    
    gridParams = {
        # step 1
#     'learning_rate': [0.01,0.05,0.1],
#     'boosting_type':['gbdt','goss'],
#     'n_estimators': [50,200,500],
#     'num_iterations':[200,400,1000],
        # step 1 fixed
    'learning_rate': [0.1], ### 0.1
    'boosting_type':['gbdt'], ### goss>gbdt
    'n_estimators': [500],
    'num_iterations':[2000], ### 2000
#     'max_depth':[6], ### <6
        # step 2
#     'num_leaves': [200], ### <400<675
#     'min_data_in_leaf':[18,20,22], ### 20 default
#     'max_bin':[127,255,511],
        # step 2 fixed
#     'num_leaves': [800],
    'max_bin':[256],
        # step 3
#     'max_depth':[7,8,9,10], ### missed
    'colsample_bytree' : [0.9], ### 0.75
    'subsample_freq':[1], ### 1
        
#     'subsample' : [0.7], ### 1
#     'min_data_in_leaf':[26],
#     'early_stopping_round':[2],
#     'reg_alpha' : [1e-3], ### 0
#     'reg_lambda' : [0,0.1,0.5], ### 0
    }
    
    if replace is True:
        gridParams = {}
    if new_params is not None:
        gridParams.update(new_params)
    

    if tuning:
        grid = GridSearchCV(clf, gridParams,
                        scoring='accuracy',
                        verbose=3,
                        cv=5,
                        n_jobs=1)
        print('default params\n',clf.get_params())
        grid.fit(Xtrain,Ytrain)
        return grid
    else:
        if not default:
            arg_str = ''
            for k,v in gridParams.items():
                if type(v[0])==str:
                    arg_str += k+'='+"'"+v[0]+"',"
                else:
                    arg_str += k+'='+str(v[0])+","
            eval(
                'clf.'+clf.set_params.__name__+"("
                    +arg_str.rstrip(',')+
                    ")"
                )
#         clf.class_weight = {1:sum(Ytrain==1),0:sum(Ytrain==0)}
        print('params\n',clf.get_params())
        clf.fit(Xtrain,Ytrain)
        Ypred = clf.predict(Xtest)
        score_train = clf.score(Xtrain,Ytrain)
        print('train score %f'%score_train)
        return [Ypred,Ytest,score_train,clf]

In [ ]:
def LGBFit(X_train,X_test,Y_train,Y_test):
    lgb_train = lgb.Dataset(X_train, Y_train)
    lgb_test = lgb.Dataset(X_test, Y_test, reference=lgb_train)

    params = {    
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'roc-auc',
#                 'nthread':6,
                'learning_rate':0.08,
                'num_leaves':300, 
                'max_depth': -1,   
                'subsample': 0.9, 
                'colsample_bytree': 0.9, 
#                 'feature_fraction': 1,
#                 'bagging_freq': 8,
# #                 'num_iterations':300,
#                 'min_data_in_leaf':2,
#                 'bagging_fraction': 0.8,
#                 'num_boost_round':3000,
            }

    cv_results = lgb.cv(params, lgb_train, nfold=5  
                        ,stratified=False, shuffle=True
                        ,seed=seed,
                        metrics=['auc','binary_logloss','mae']
                        ,verbose_eval=1)
    print('best n_estimators:', len(cv_results['auc-mean']))
    for k,v in cv_results.items():
        print('best cv score:', k, pd.Series(cv_results[k]).max())
    return [lgb,cv_results]

#### main

In [ ]:
'''
'data_NPInter10412',
'reRPI2825',
'RPI488',
'RPI2241',
'RPITER_RPI1807'
'''

hyper_params = GetConfigure()
num_hyper_params = len(hyper_params)
lassocv_param = {'threshold':0.05}

cv = 5
generalize_ratio = 1.0/cv
test_ratio = 1.0/cv

mi_use = False
outside_grid = True
tuning_mode = False

if tuning_mode:
    cv = 1

cv_results = []
# scores = []
'''
    topK =3000
'''


search_param = [{'learning_rate':[0.05,0.1,0.01,0.02],
                'colsample_bytree':[0.7,0.8,0.9,1],
                 'max_depth':[5,6,7,8,9],
                 'num_iterations':[500,1000,2000],
#                  'min_data_in_leaf':[20,25,30]
                }]
search_grid = list(ParameterGrid(search_param))

conf_param = {}


cfile = GetConfigureObject()
commons = dict(cfile.items('common'))

db = redis.StrictRedis(host=commons[str.lower('REDIS_HOST')], port=6379, db=0, 
                      password=commons[str.lower('REDIS_PWD')], decode_responses=True)

In [ ]:
[data,T] = ReadData()



for batch in range(cv):
    INFO('cross validation batch %d'%batch)
    if mi_use==True:
        arr = ToMatrix(data,'sparse')
        [X_train,X_test,Y_train,Y_test] = MutualInformationFeatureSelection2(arr,data,generalize_ratio)
        [X_train,X_test,Y_train,Y_test] = \
            RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test)
    else:
        [X,Y] = ToMatrix(data,'dense')
        [X_train,X_test,Y_train,Y_test] = SplitDataset(X,Y,generalize_ratio)
#         X_train,X_test,Y_train,Y_test = IsomapDimensionalityReduction(X_train,X_test,Y_train,Y_test)
#         [X_train,X_test,Y_train,Y_test] = LassoCVFeatureSelection(X_train,X_test,Y_train,Y_test,lassocv_param)
        [X_train,X_test,Y_train,Y_test] = \
            RandomForestDimensionalityReduction(X_train,X_test,Y_train,Y_test)
#         [X_train,X_test,Y_train,Y_test] = \
#             PCADimensionalityReduction(X_train,X_test,Y_train,Y_test)
    if tuning_mode:
        [Xtrain,Ytrain] = merge_train_test(X_train,X_test,Y_train,Y_test)
        grid = LGBTuning(Xtrain,[],Ytrain,[],True)
        cv_results.append(grid)
        break
    else:
        if outside_grid is True:
            for sp in search_grid:
                sp = dict(map(lambda x:(x,[sp[x]]),sp))
                tune_results = LGBTuning(X_train,X_test,Y_train,Y_test,sp,tuning=False)
                r = {'batch':batch,'res':tune_results,'sp':sp,'score':scoreFunction(tune_results)}
                cv_results.append(r)
            break
        else:
            tune_results = LGBTuning(X_train,X_test,Y_train,Y_test,tuning=False,default=False)
            r = {'batch':batch,'res':tune_results,'score':scoreFunction(tune_results)}
            cv_results.append(r)
        


In [ ]:
### compute scores
scores = [scoreFunction(r['res'])for r in cv_results]
auc = np.mean([x['auc'] for x in scores])
acc = np.mean([x['acc'] for x in scores])
scores,auc,acc

In [ ]:
cv_results

In [ ]:

### save model
if outside_grid is True:
    search_param = set( list(map(lambda x:x['sp'],cv_results)) )
    scores = [{sp:[] for sp in search_param}]
    for sp in search_param:
        scores[sp] = np.mean(list( map(lambda x:x['score'],filter(lambda x:x['sp']==sp,cv_results)) ))
    best_sp = sorted(scores,key=lambda x:scores[x][0],reverse=True)[0]['sp']
    


#### test

In [ ]:
# DecisionTreePrefit(X_train,X_test,Y_train,Y_test)
# [lgb,cv_results] = LGBFit(X_train,X_test,Y_train,Y_test)
# WriteResult(DATASET,cv_results,conf,commons)
np.mean(scores['acc'][-10:])

#### tuning

In [ ]:
scores = [score for grid in cv_results for score in grid.cv_results_['mean_test_score']]
sns.distplot(scores,rug=True,bins=20)
plt.show()

# param_rank = np.array([grid.cv_results_['mean_test_score'],grid.cv_results_['params']]).T
# a = sorted(param_rank,key=lambda x:x[0],reverse=True)
# a = np.array(list(a))

# a
max( cv_results,key=lambda x:np.mean(x.cv_results_['mean_test_score']) ).best_params_

In [ ]:
feature_importance = list( sorted(grid.best_estimator_.feature_importances_,reverse=True) )
sum(feature_importance[:2000])/sum(feature_importance)

plt.figure()
sns.distplot(grid.best_estimator_.feature_importances_,bins=100)
plt.xlabel('feature importance')
plt.ylabel('ratio')
plt.show()